## Modelo Baseline con la media
El objetivo de este notebook es realizar un modelo básico con la media del precio del gas para poder compararlo con el modelo de series temporales y sacar conclusiones

In [ ]:
import pandas as pd
import numpy as np

from prophet import Prophet

from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
# Si usas colab usar este scrip
df = pd.read_csv("/content/SET_INPUTS_BASE_0.csv", sep=';')

In [ ]:
# Si usas VSC u otro usar este scrip
# df = pd.read_csv("Poner tu ruta del archivo aquí")
# df.head()

In [ ]:
df["ds"] = pd.to_datetime(df[["Year", "Month", "Day"]])

df = df[~((df["ds"].dt.month == 2) & (df["ds"].dt.day == 29))].copy()

df_prophet = df[["ds", "Price_Gas_EUR_MWh"]].rename(
    columns={"Price_Gas_EUR_MWh": "y"}
)

In [ ]:
# Poner las columnas como STR
cols = ['Demanda_GWh', 'Gen_Eolica_GWh', 'Gen_Solar_GWh', 'Price_Elec_EUR_MWh', 'Price_Gas_EUR_MWh']
df[cols] = df[cols].astype(str).apply(lambda x: x.str.replace(',', '.', regex=False))
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
train = df[(df["ds"] >= "2019-01-01") & (df["ds"] < "2023-01-01")].copy()
test  = df[(df["ds"] >= "2023-01-01") & (df["ds"] < "2023-12-31")].copy()


In [ ]:
mean_value = train["Price_Gas_EUR_MWh"].mean()

test["baseline_mean"] = mean_value

43.33756506849315


In [ ]:
df_prophet = train[["ds", "Price_Gas_EUR_MWh"]].rename(columns={"Price_Gas_EUR_MWh": "y"})

from prophet import Prophet

model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode="additive"
)
model.fit(df_prophet)

future = model.make_future_dataframe(periods=len(test), freq="D")
forecast = model.predict(future)

forecast_test = forecast[forecast["ds"].isin(test["ds"])].copy()
test = test.merge(forecast_test[["ds", "yhat"]], on="ds")


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae_baseline = mean_absolute_error(test["Price_Gas_EUR_MWh"], test["baseline_mean"])

rmse_baseline = np.sqrt(mean_squared_error(test["Price_Gas_EUR_MWh"], test["baseline_mean"]))

wape_baseline = (
    (test["Price_Gas_EUR_MWh"] - test["baseline_mean"]).abs().sum()
    / test["Price_Gas_EUR_MWh"].abs().sum()
) * 100


print("MAE Baseline:", mae_baseline, "€")
print("RMSE Baseline:", rmse_baseline, "€")
print("WAPE Baseline:", wape_baseline, "%")

MAE Baseline: 8.993876900496762 €
RMSE Baseline: 10.545382508295068 €
WAPE Baseline: 22.92699749340695 %
